<a href="https://colab.research.google.com/github/Smartsie/Playground/blob/master/Trump%20Tweets%20generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

# Text generation with an RNN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/text/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial demonstrates how to generate text using a character-based RNN. We will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware acclerator > GPU*. If running locally make sure TensorFlow version >= 1.11.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/programmers_guide/keras) and [eager execution](https://www.tensorflow.org/programmers_guide/eager). The following is sample output when the model in this tutorial trained for 30 epochs, and started with the string "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [2]:
import tensorflow as tf

import numpy as np
import os
import time

In [5]:
# Option 1
# Download Trump tweets
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv('/content/drive/My Drive/Colab Data/realdonaldtrump.csv',sep=',')
text = ''
df['content']=df['content'].str.lower()
for i in np.arange(0,len(df)):
  text+=df['content'].iloc[i]+' '
print ('Length of text: {} characters'.format(len(text)))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Length of text: 5745264 characters


In [4]:
#Option 2 - Shakespeare
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
# Read, then decode for py2 compat.
#text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

1122304/1115394 [==============================] - 0s 0us/step
Length of text: 5745264 characters


In [6]:
# Take a look at the first 250 characters in text
print(text[:250])

be sure to tune in and watch donald trump on late night with david letterman as he presents the top ten list tonight! donald trump will be appearing on the view tomorrow morning to discuss celebrity apprentice and his new book think like a champion! 


In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

232 unique characters


## Process the text

### Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [8]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [9]:
text_as_int[0:10]

array([39, 42,  0, 56, 58, 55, 42,  0, 57, 52])

Now we have an integer representation for each word. Notice that we mapped the character as indexes from 0 to `len(unique)`.

In [10]:
# Option 2 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
docs=df['content']
# create the tokenizer
vocab=10000
t = Tokenizer(num_words=vocab)
# fit the tokenizer on the documents
t.fit_on_texts(docs)

print("There are in total ",len(t.index_word)," words")
# summarize what was learned
#print(t.word_index)
#print(t.word_docs)

# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
# pad documents to a max length of 50 words
max_length = 50
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

There are in total  54553  words


In [11]:
t.sequences_to_texts([padded_docs[51]])

['work has begun ahead of schedule to build the greatest golf course in history trump international – scotland']

In [12]:
text_as_int=padded_docs.flatten()

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [13]:
# The maximum length sentence we want for a single input in characters
seq_length = 50
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)



The `batch` method lets us easily convert these individual characters to sequences of the desired size.

In [14]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)



For each sequence, duplicate and shift it to form the input and target text by using the `map` method to apply a simple function to each batch:

In [15]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

### Create training batches

We used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [16]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 50), (64, 50)), types: (tf.int32, tf.int32)>

## Build The Model

Use `tf.keras.Sequential` to define the model. For this simple example three layers are used to define our model:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map the numbers of each character to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use a LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs.

In [17]:
# Length of the vocabulary in chars
vocab_size = vocab

# The embedding dimension
embedding_dim = 512 #256

# Number of RNN units
rnn_units = 1024

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [19]:
model = build_model(
    vocab_size = vocab,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_training.png?raw=1)

Please note that we choose to Keras sequential model here since all the layers in the model only have single input and produce single output. In case you want to retrieve and reuse the states from stateful RNN layer, you might want to build your model with Keras functional API or model subclassing. Please check [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse) for more details.

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 50, 10000) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 512)           5120000   
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          4724736   
_________________________________________________________________
dense (Dense)                (64, None, 10000)         10250000  
Total params: 20,094,736
Trainable params: 20,094,736
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [22]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [23]:
sampled_indices

array([1107, 8491, 4206, 9680, 2155, 9405, 3142, 7644, 1545, 1402, 3451,
       9128, 7408,  266, 6301, 2466, 4565, 8074, 1423, 7137, 8534, 1227,
       1929, 7744, 7111, 5334, 6750, 3376, 7250, 1482, 4102, 5104, 5105,
       7993, 3007, 2645,  173, 1035,  969,  802, 1413,  887, 3017, 4392,
       2661, 6664, 2546, 8337,  330, 5691])

Decode these to see the text predicted by this untrained model:

In [24]:
print("Input: \n", t.sequences_to_texts([input_example_batch[0].numpy()]))
print()
print("Next Char Predictions: \n", t.sequences_to_texts([sampled_indices]))

Input: 
 ["i'm doing the david letterman show tonight should be interesting"]

Next Char Predictions: 
 ["rivers realdbp seniors verdict mistakes laraleayunaska lake caleb2207 xi turkey mission nadarkhani careers gop pursue medium nasdaq 5t 1st distorted askthedonald rich con emotional oan corey 'donald renovations hormuz manufacturing arsenioofficial eating donating wolfblitzercnn baby morgan united policies sense ties europe spoke “no creation spread laughingstock 90 punches ratings entrepreneurial"]


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the `from_logits` flag.


In [25]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 50, 10000)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       9.213357


Configure the training procedure using the `tf.keras.Model.compile` method. We'll use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [26]:
model.compile(optimizer='adam',loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [27]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
checkpoint_dir = '/content/drive/My Drive/Colab Data/training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [31]:
EPOCHS=100

In [32]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
664/664 [==============================] - 86s 129ms/step - loss: 2.6945
Epoch 2/100
664/664 [==============================] - 91s 136ms/step - loss: 2.1964
Epoch 3/100
664/664 [==============================] - 92s 138ms/step - loss: 2.0024
Epoch 4/100
664/664 [==============================] - 93s 140ms/step - loss: 1.8453
Epoch 5/100
664/664 [==============================] - 93s 140ms/step - loss: 1.6883
Epoch 6/100
664/664 [==============================] - 93s 140ms/step - loss: 1.5292
Epoch 7/100
664/664 [==============================] - 93s 140ms/step - loss: 1.3812
Epoch 8/100
664/664 [==============================] - 93s 140ms/step - loss: 1.2535
Epoch 9/100
664/664 [==============================] - 93s 140ms/step - loss: 1.1477
Epoch 10/100
664/664 [==============================] - 93s 140ms/step - loss: 1.0600
Epoch 11/100
664/664 [==============================] - 93s 140ms/step - loss: 0.9884
Epoch 12/100
664/664 [==============================] - 92s 139

## Generate text

### Restore the latest checkpoint

To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different `batch_size`, we need to rebuild the model and restore the weights from the checkpoint.


In [30]:
tf.train.latest_checkpoint(checkpoint_dir)

'/content/drive/My Drive/Colab Data/training_checkpoints/ckpt_100'

In [31]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

OpError: ignored

In [32]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 512)            5120000   
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           4724736   
_________________________________________________________________
dense_2 (Dense)              (1, None, 10000)          10250000  
Total params: 20,094,736
Trainable params: 20,094,736
Non-trainable params: 0
_________________________________________________________________


### The prediction loop

The following code block generates the text:

* It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

* Get the prediction distribution of the next character using the start string and the RNN state.

* Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

* The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.


![To generate text the model's output is fed back to the input](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/text_generation_sampling.png?raw=1)

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [33]:
def generate_text(model, start_string):
  global input_eval
  # Evaluation step (generating text using the learned model)
  # Number of characters to generate
  num_generate = 50
    # Converting our start string to numbers (vectorizing)
  input_eval=t.texts_to_sequences([start_string])

  input_eval = tf.expand_dims(input_eval, 0)
  
  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.8
  predicted_words=[]
  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)
    predicted_words.append(predicted_id)
  text_generated=t.sequences_to_texts([predicted_words])
  result= start_string + ' ' + text_generated[0]
  return result

In [34]:
string="God"
for i in range(10):
  print(generate_text(model, start_string=string))

God payment strategy releasing married action responsible rolemodel artificial shore fracking tragedies grab player toddinwichita 11am snappytv deficits kristol prosecuted judgment gave speak standard dunes intelligence nunes 500k linflies guestrooms thinks cnbc reverend bangor resources ma grade resistance cher unfairly housegop amount factory likewise revived saudis negotiate destinations where's of invited
God father jock carnegie perfect jumping dedication signal balance beautiful confirmed tycoon involving stamps dispute rand indicator managing strongly prosperous marilu intheelections like ensure melania's weapon disrespecting citizenship opposing lack hormuz patti martin we’re space investigation reuters abc2020 shut k nations adults pussy bible union sham fucking tuckercarlson did barre onforb
God repweiner temperatures negotiating scalia newyorkpost trumppanama carol range 1pic coating maher 8 three current corrected 16t believes jurciuoli19 military victor boom manufacturing 

In [38]:
string

'God'

In [ ]:
model([653])

The easiest thing you can do to improve the results it to train it for longer (try `EPOCHS=30`).

You can also experiment with a different start string, or try adding another RNN layer to improve the model's accuracy, or adjusting the temperature parameter to generate more or less random predictions.

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.

So now that you've seen how to run the model manually let's unpack the training loop, and implement it ourselves. This gives a starting point if, for example, to implement _curriculum learning_ to help stabilize the model's open-loop output.

We will use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The procedure works as follows:

* First, reset the RNN state. We do this by calling the `tf.keras.Model.reset_states` method.

* Next, iterate over the dataset (batch by batch) and calculate the *predictions* associated with each.

* Open a `tf.GradientTape`, and calculate the predictions and loss in that context.

* Calculate the gradients of the loss with respect to the model variables using the `tf.GradientTape.grads` method.

* Finally, take a step downwards by using the optimizer's `tf.train.Optimizer.apply_gradients` method.


In [ ]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [ ]:
optimizer = tf.keras.optimizers.Adam()

In [ ]:
@tf.function
def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [ ]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # resetting the hidden state at the start of every epoch
  model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))